In [1]:
from datasets import load_dataset

training_data = load_dataset("financial_phrasebank", "sentences_allagree")
valitdation_data = load_dataset("financial_phrasebank", "sentences_75agree")

In [2]:
from transformers import  TFGPT2Tokenizer

tokenizer =  TFGPT2Tokenizer.from_pretrained("gpt2")

def preprocess_function(examples):
    return tokenizer(examples["sentence"])

train_tokens = training_data.map(preprocess_function)



2023-11-29 15:21:53.902361: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-29 15:21:54.024139: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-29 15:21:54.024170: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-29 15:21:54.024599: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-29 15:21:54.087447: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

Using TensorFlow backend


Parameter 'function'=<function preprocess_function at 0x7f8fdd1fb6d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/2264 [00:00<?, ? examples/s]

In [74]:
val_tokens = valitdation_data.map(preprocess_function)
val_tokens

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'attention_mask', 'input_ids'],
        num_rows: 3453
    })
})

In [3]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 3
batches_per_epoch = len(train_tokens["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

batches_per_epoch

141

In [50]:
print(train_tokens["train"][0])

{'sentence': 'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .', 'label': 1, 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'input_ids': [4821, 284, 17113, 837, 262, 1664, 468, 645, 3352, 284, 1445, 477, 3227, 284, 3284, 837, 3584, 326, 318, 810, 262, 1664, 318, 3957, 764]}


In [4]:
id2label = {0: "negative", 2: "positive", 1: "neutral"}
label2id = {"negative": 0, "positive": 2,"neutral": 1}

In [5]:
from transformers import  TFGPT2ForSequenceClassification

model =  TFGPT2ForSequenceClassification.from_pretrained(
    "gpt2", num_labels=3, id2label=id2label, label2id=label2id, problem_type="multi_label_classification"
)

All PyTorch model weights were used when initializing TFGPT2ForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFGPT2ForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
tf_train_set = model.prepare_tf_dataset(

    train_tokens["train"],

    shuffle=True,

    batch_size=1,
)

"""
tf_validation_set= model.prepare_tf_dataset(

    val_tokens["train"],

    shuffle=True,

    batch_size=1,
)
"""

'\ntf_validation_set= model.prepare_tf_dataset(\n\n    val_tokens["train"],\n\n    shuffle=True,\n\n    batch_size=1,\n)\n'

In [35]:
tf_train_set

<_PrefetchDataset element_spec=({'attention_mask': TensorSpec(shape=(1, None), dtype=tf.int64, name=None), 'input_ids': TensorSpec(shape=(1, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(1,), dtype=tf.float32, name=None))>

In [7]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

In [84]:
import evaluate

accuracy = evaluate.load("accuracy")

import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
from transformers import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="chat2Fsentiment", tokenizer=tokenizer, hub_model_id="Dave12121/chat2Fsentiment"
)

/home/gm/Documents/unb-course/NLP/chat2Fsentiment is already a clone of https://huggingface.co/Dave12121/chat2Fsentiment. Make sure you pull the latest changes with `repo.git_pull()`.


In [10]:


callbacks = [push_to_hub_callback]

## Cannot handle batch sizse bigger then one if no padding token is defiened

In [11]:
model.fit(x=tf_train_set, callbacks=callbacks,epochs=3)

Epoch 1/3
2264/2264 [==============================] - ETA: 0s - loss: 0.6606

AttributeError: 'TFGPT2Tokenizer' object has no attribute 'save_pretrained'

In [62]:
sample =  "The company still expects its turnover in 2010 to slightly increase from the level of 2009 , adding that `` market predictability is still too poor for trustworthy forecasts on the market development of the contract manufacturing business during the current year '' ."
#@positive 


res = tokenizer(sample)

res

{'attention_mask': <tf.Tensor: shape=(45,), dtype=int32, numpy=
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1], dtype=int32)>,
 'input_ids': <tf.Tensor: shape=(45,), dtype=int32, numpy=
 array([  464,  1664,   991, 13423,   663, 25079,   287,  3050,   284,
         4622,  2620,   422,   262,  1241,   286,  3717,   837,  4375,
          326,  7559,  1910,  4331,  1799,   318,   991,  1165,  3595,
          329, 34412, 26119,   319,   262,  1910,  2478,   286,   262,
         2775,  9138,  1597,  1141,   262,  1459,   614, 10148,   764],
       dtype=int32)>}

In [73]:
result = model({"input_ids": res['input_ids'], "attention_mask": tf.expand_dims(res['attention_mask'], axis=0)}).logits

print(result)

predicted_class_id = int(tf.math.argmax(result, axis=-1)[0])

print(predicted_class_id)

tf.Tensor(
[[-0.4324854   3.0709777  -0.40611345]
 [-2.4569297   8.503071   -1.7715104 ]
 [-3.3441517   9.665132   -2.3678463 ]
 [-1.8486593   6.0433984  -1.1951487 ]
 [-2.1100042   9.485331   -1.5617089 ]
 [-1.791574    6.8757706  -0.5577465 ]
 [-1.8085469   8.50463    -1.4421874 ]
 [-1.4257947   4.8182955  -0.57911605]
 [-2.3357275   9.129903   -1.8939478 ]
 [-2.0574517   9.119795   -1.3780642 ]
 [-2.4704866   8.949904   -0.86589247]
 [-2.8590393  11.990199   -1.5792216 ]
 [-1.851683   10.738962   -1.3899277 ]
 [-2.6057053   8.11302    -1.1163359 ]
 [-2.7843194  12.061036   -1.5193974 ]
 [-1.5283071   5.541763   -0.18603367]
 [-2.378657   10.171539   -0.7288166 ]
 [-1.8800042   8.486394   -1.0343827 ]
 [-2.2138212   8.6564455  -1.4366947 ]
 [-2.2832594   9.216361   -0.17686924]
 [-1.0274462   8.656375   -0.5509918 ]
 [-1.3884641   6.5475698  -0.3847338 ]
 [-1.976958    7.039391   -1.4013064 ]
 [-2.5766757  11.581657   -2.0108888 ]
 [-2.2498922  11.48768    -2.006533  ]
 [-2.0672467  

In [12]:
model.save_pretrained("chatGPT2-instance", from_tf=True)
#tokenizer.save_pretrained("chatGPT", from_tf=True)

In [14]:
model.save_pretrained("chatGPT2-instance-nontf")
